<a href="https://colab.research.google.com/github/edoardorossi1995/Tesi/blob/main/Average_Reward_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pseudocodice 
###Algoritmo di tipo Q-learning, off policy, a task non episodico, che approssima la funzione valore con una rete neurale, con un framework di tipo average reward.






#Drive Setting


In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB == True:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd /content/gdrive/MyDrive/Tesi
  #!ls

import warnings
import pickle
import sys
sys.path.insert(0, '/content/gdrive/MyDrive/Tesi/functions')
sys.path.insert(0, '/content/gdrive/MyDrive/Tesi')

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense, Lambda
from keras.activations import exponential, relu
from keras import backend as K

from pkl import store_data, load_data
from functions.compressor import compress

import random
from collections import defaultdict

Mounted at /content/gdrive
/content/gdrive/MyDrive/Tesi


# Import Anagrafica e CTA

In [3]:
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt

CTA30_def = ['/content/gdrive/MyDrive/Tesi/CTA30/DF_clean/clean_CTA_30_def.csv']
df_def = pd.read_csv(CTA30_def[0])

CTA30_18 = ['/content/gdrive/MyDrive/Tesi/CTA30/DF_clean/clean_CTA_30_2018.csv']
df = pd.read_csv(CTA30_18[0])
df['VLV_RAF_FDBK_00'] = df['VLV_RAF_FDBK_00'] / 100
df['VLV_SUR_FDBK_00'] = df['VLV_SUR_FDBK_00'] / 100
scaler = MinMaxScaler()
act_f_m = df.loc[:,'F_M_FDBK_00']
a_f_m_reshaped = np.array(act_f_m).reshape(-1, 1)
a_f_m_scaled = scaler.fit_transform(a_f_m_reshaped)

pca_df = compress(df)

In [4]:
print("Anagrafica shape:",df_def.shape,"\nDataframe shape:", df.shape,"\nPca Dataframe shape:", pca_df.shape)

Anagrafica shape: (76, 2) 
Dataframe shape: (141811, 76) 
Pca Dataframe shape: (141811, 7)


In [5]:
df_def

,pointColName,pointName
0,AUTOMAN_SERR_CMD_00,Automatico Manuale Serrande Comando
1,AUTOMAN_VLV_CMD_00,Automatico Manuale Valvole Comando
2,AUTOMAN_M_CMD_00,Automatico Manuale Mandata Comando
3,AUTOMAN_R_CMD_00,Automatico Manuale Ripresa Comando
4,LOCREM_CMD_00,Selettore Locale Remoto Comando
...,...,...
71,VOLT_R_00,Tensione Ripresa
72,T_M_SP_00,Temperatura Aria Mandata Setpoint
73,T_R_SP_00,Temperatura Aria Ritorno Setpoint
74,T_AMB_04,Temperatura Aria Ambiente Media


# Rete Neurale

Log-sum-exp neural networks and posynomial
models for convex and log-log-convex data 

(Calafiore, Gaubert, Possieri)

In [6]:
n = load_data('*.pkl')
action_vector_dimension = 2

# Funzione di attivazione logatirmica (ultimo strato)
def log_act(x):
  return K.log(x)

# Architettura rete neurale
input_dim = n['n_components']+action_vector_dimension
inner_dim_1 = 32
output_dim = 1

print("input dim: ", input_dim)
print("inner layer dim: ", inner_dim_1)
print("output dim: ", output_dim)

# Costruzione strati
rete_neurale = Sequential()
rete_neurale.add(Dense(units = inner_dim_1, activation = exponential, input_dim = input_dim))
rete_neurale.add(Dense(units = output_dim, activation=log_act)) 

rete_neurale.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

input dim:  9
inner layer dim:  32
output dim:  1


In [38]:
from scipy.optimize import minimize_scalar

# Definisci la funzione che restituisce l'output della rete neurale
def neural_net_output(x, s):
    # Concatena l'array s con l'array a
    a = x.reshape(-1, 2)
    inp = np.concatenate((s, a), axis=1)
    
    # Restituisci l'output della rete neurale
    # Ad esempio, utilizzando Keras:
    return rete_neurale.predict(inp)
    

# Definisci la funzione obiettivo da massimizzare
def objective_function(x, s):
  
    # Calcola l'output della rete neurale
    output = neural_net_output(x, s)
    
    # Restituisci la somma degli output della rete neurale
    return -output

# Definisci l'array s e l'array a
s = np.array([[1, 2, 3, 4, 5, 6, 7]])
a0_t = df.loc[:,'F_M_FDBK_00']
a0_t_scaled = a_f_m_scaled
a1_t = df.loc[:,'VLV_RAF_FDBK_00']
a = np.array([a0_t_scaled, a1_t])

# Scegli un punto iniziale per le variabili dell'array a
x0 = np.random.rand(200)

# Esegui la ricerca dell'ottimo
res = minimize(objective_function, x0, args=(s,), method='Nelder-Mead')

# Estrai le variabili ottimali dell'array a
x_opt = res.x
a_opt = x_opt.reshape(-1, 2)

# Calcola il nuovo vettore di input per la rete neurale
inp_opt = np.concatenate((s, a_opt), axis=1)

# Calcola l'output massimo della rete neurale corrispondente al vettore di input ottimale
output_max = np.max(neural_net_output(x_opt, s))

IndexError: ignored

# Q-Learning Agent

In [13]:
# Dimensione problema e inizializzazione stato iniziale
[S, feat] = df.shape
current_state = 1500#np.random.randint(0,S-1)

# Inizializza la media delle ricompense future attese per ogni stato-azione a zero
q = 0

# Stato corrente
s = current_state

# Parametri di apprendimento (learning rate, discount factor, average rate)
alpha = 0.5
gamma = 1
beta = 0.5

# Media mobile esponenziale (avr = avr + beta * (r - avr)) 
# [scelta perché computazionalmente più leggera di una media aritmetica]
average_reward = 0

# Setting Episodi
total_episodes = 3
current_episode = 0

# Ripeti finché non raggiungi la fine del processo
for current_episode in range(total_episodes):
    
    # Stato successivo e ottieni l'azione A e la ricompensa R
    s_prime = s+1
    if s_prime > S-1:
      print("s =",s)
      break

    #a1 = df.loc[s,'F_M_FDBK_00']-df.loc[s_prime,'F_M_FDBK_00']
    a1 = df.loc[s,'F_M_FDBK_00']
    #a2 = df.loc[s,'VLV_RAF_FDBK_00']-df.loc[s_prime,'VLV_RAF_FDBK_00']
    a2 = df.loc[s,'VLV_RAF_FDBK_00']
    a = (a1,a2)
    r = -df.loc[s,'PW_ATT_M_00']

    average_reward = average_reward + beta * (r - average_reward)

    # Comprimo s_prime per generare un valore di input_prime per la predizione di q(s',a')
    pca_s_prime = pca_df.loc[s_prime,:]
    a1_prime = df.loc[s_prime,'F_M_FDBK_00']
    a2_prime = df.loc[s_prime,'VLV_RAF_FDBK_00']
    a_prime = np.array([[a1_prime, a2_prime]])
    print("shape", a1_prime, a2_prime)
    print("complete", a_prime)
    a_prime.reshape(1,-1)
    a_prime = scaler.transform(a_prime)
    
    nn_input_prime = np.concatenate((pca_s_prime.values, np.array([a_prime[0],a_prime[1]])))
    nn_input_prime = np.reshape(nn_input_prime, (1,input_dim))

    # Calcola la media delle ricompense future attese per lo stato-azione corrente
    # q = q + a*(avr + gamma*max(q') - q)
    print("pre q(s,a) = ",q)
    print("nn_input_prime = ",nn_input_prime)
    if current_episode != 0:
      #print("nn = ",rete_neurale.predict(nn_input_prime))
      q_prime = rete_neurale.predict(nn_input_prime)[0]
      print("q'.size = ",q_prime.size, "\nq' = ", q_prime)
      q = q + alpha * (average_reward + gamma * np.max(q_prime) - q)
    print("q(s,a) = ",q)

    # Calcola il valore target
    target = q
    
    # Comprimo lo stato e genero l'input della rete neurale
    pca_s = pca_df.loc[s,:]

    nn_input = np.concatenate((pca_s.values, np.array([a[0],a[1]])))
    nn_input = np.reshape(nn_input, (1,input_dim))
    target = np.array([target])

    # Aggiorna la rete neurale
    rete_neurale.fit(nn_input, target, epochs=1, verbose=0)

    # Aggiorna lo stato corrente
    s = s_prime

    # Plot statistiche ogni 100 episodi
    if current_episode%1 == 0:
      print("Episodio ",current_episode,"/",total_episodes)
      print("target: ", target, "\nreward: ",r,"\naverage_reward: ",np.mean(average_reward))
      print("\n")

# Restituisci la rete neurale ottenuta
#return rete_neurale

shape 100.0436 0.0
complete [[100.0436   0.    ]]


ValueError: ignored